## Evaluate buildings and places around a real estate listing in Somerville, MA

In [19]:
from pyplaces import overture_maps as om
import folium

In [20]:
def plot_geodataframes(gdfs, zoom_start=15, popup_columns=None):
    if not isinstance(gdfs, list):
        gdfs = [gdfs]
    if popup_columns is None:
        popup_columns = [None] * len(gdfs)

    # Filter out empty GeoDataFrames
    gdfs_filtered = [(gdf, popup_columns[i]) for i, gdf in enumerate(gdfs) if not gdf.empty]

    if not gdfs_filtered:
        print("All GeoDataFrames are empty.")
        return None

    # Center the map at the centroid of the first non-empty GeoDataFrame
    center = gdfs_filtered[0][0].geometry.unary_union.centroid
    m = folium.Map(location=[center.y, center.x], zoom_start=zoom_start, tiles="CartoDB positron")

    for gdf, popup_col in gdfs_filtered:
        # Check if all geometries are Points
        if gdf.geometry.geom_type.isin(['Point']).all():
            for _, row in gdf.iterrows():
                coords = [row.geometry.y, row.geometry.x]
                popup = str(row[popup_col]) if popup_col and popup_col in row else None
                folium.CircleMarker(
                    location=coords,
                    radius=1,   # size of the dot
                    color='red',
                    fill=True,
                    fill_color='red',
                    fill_opacity=1,
                    popup=popup
                ).add_to(m)
        else:
            # Otherwise treat as polygon or line geometries
            if popup_col and popup_col in gdf.columns:
                folium.GeoJson(
                    gdf,
                    tooltip=folium.GeoJsonTooltip(fields=[popup_col], aliases=['Name:']),
                    popup=folium.GeoJsonPopup(fields=[popup_col])
                ).add_to(m)
            else:
                folium.GeoJson(gdf).add_to(m)

    return m


From Overture Maps, we can fetch the places and buildings within a half mile radius around address an address that we are interested in.

In [21]:
#Get the buildings and places around this address in Somerville
address = "72 Holland St, Somerville, MA"
gdf_buildings = om.overture_buildings_from_address(address,distance=0.5,unit="mi")
gdf_places = om.overture_places_from_address(address,distance=0.5,unit="mi")

#Since the names of a place are in a dictionary object, take out the primary name and put it into a new column for each place.
gdf_places['primary_name'] = gdf_places['names'].apply(lambda x: x.get('primary') if isinstance(x, dict) else None)

#Show places
gdf_places.head(5)

,id,geometry,bbox,version,sources,names,categories,confidence,websites,socials,emails,phones,brand,addresses,primary_name
0,08f2a306708e3c95036395d446f94fb5,POINT (-71.13369 42.39108),"{'xmin': -71.13369750976562, 'xmax': -71.13368...",0,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': 'Carriegustafsonglass', 'common': ...","{'primary': 'glass_blowing', 'alternate': ['ar...",0.472681,[http://Www.carriegustafson.com/],[https://www.facebook.com/504041206291258],None,[+17813674024],None,"[{'freeform': '147 Sherman St', 'locality': 'C...",Carriegustafsonglass
1,08f2a306708e3c9503d386ef500d8fda,POINT (-71.13322 42.39102),"{'xmin': -71.13321685791016, 'xmax': -71.13320...",0,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': 'Almost Perfect Glass Inc', 'commo...","{'primary': 'glass_and_mirror_sales_service', ...",0.978172,[http://www.almostperfectglass.com/],[https://www.facebook.com/339758332833589],None,[+16174928784],None,"[{'freeform': '147 Sherman St', 'locality': 'C...",Almost Perfect Glass Inc
2,08f2a306708e02c00360c0fc4cc07f8b,POINT (-71.13306 42.39112),"{'xmin': -71.13306427001953, 'xmax': -71.13305...",0,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': 'Sherman St Condo Tr', 'common': N...",{'primary': 'landmark_and_historical_building'...,0.876457,None,[https://www.facebook.com/126692407970572],None,[+16175769479],None,"[{'freeform': '147 Sherman St', 'locality': 'C...",Sherman St Condo Tr
3,08f2a306708e394a0342ced905a65db6,POINT (-71.133 42.39101),"{'xmin': -71.13300323486328, 'xmax': -71.13299...",0,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': 'NOCA Glass School', 'common': Non...","{'primary': 'art_school', 'alternate': ['schoo...",0.978172,[http://www.nocaglassschool.com/],[https://www.facebook.com/136564065108],None,[+16172450350],None,"[{'freeform': '147 Sherman St', 'locality': 'C...",NOCA Glass School
4,08f2a306708e40c203d132bb22b9d451,POINT (-71.13295 42.39162),"{'xmin': -71.1329574584961, 'xmax': -71.132949...",0,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': 'Abraham Watson House', 'common': ...",{'primary': 'landmark_and_historical_building'...,0.472681,[http://en.wikipedia.org/wiki/index.html?curid...,[https://www.facebook.com/108388655847892],None,None,None,"[{'freeform': None, 'locality': 'Cambridge', '...",Abraham Watson House


In [23]:
plot_geodataframes([gdf_buildings[["id","geometry","class"]],gdf_places[["id","geometry","primary_name"]]],popup_columns=["primary_name","class"])

C:\Users\tedba\AppData\Local\Temp\ipykernel_20240\2279583328.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gdfs_filtered[0][0].geometry.unary_union.centroid
